<a href="https://colab.research.google.com/github/sameer203/Tensorflow_/blob/main/11_NLP_fundamentals_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-11-02 12:50:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.187.128, 64.233.188.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.187.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-11-02 12:50:51 (99.1 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-11-02 12:50:51--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-02 12:50:51 (87.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys,unzip_data

In [4]:
unzip_data("nlp_getting_started.zip")

### Visualizing Test data
1. pandas
2. tensorflowload text

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
len(train_df), len(test_df)

(7613, 3263)

In [11]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real diaster)" if target > 0 else "(not real diaster)")
  print(f"Text:\n {text}\n")

Target: 1 (real diaster)
Text:
 @KapoKekito on northgate by the taco truck that's fire.

Target: 1 (real diaster)
Text:
 DLH issues Hazardous Weather Outlook (HWO) http://t.co/WOzuBXRi2p

Target: 1 (real diaster)
Text:
 #reuters Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/ShzPyIQok5

Target: 1 (real diaster)
Text:
 China's Stock Market Crash: Are There Gems In The Rubble? http://t.co/j4ggmKINEy #forbesasia

Target: 0 (not real diaster)
Text:
 @Safyuan just a minor citation for possesion of a decriminalized substance im not facing any time



In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(train_df_shuffled["text"].to_numpy(), 
                                                                              train_df_shuffled["target"].to_numpy(), 
                                                                              test_size=0.1, 
                                                                              random_state=42)

In [14]:
len(train_sentences), len(test_sentences)

(6851, 762)

In [15]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

### Text Vectorization

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [29]:
# use the defaultTextVectorization parameters
text_vectorization = TextVectorization(max_tokens=None,
                                       standardize="lower_and_strip_punctuation",
                                       split="whitespace",
                                       ngrams=None,
                                       output_mode="int",
                                       output_sequence_length=None,
                                       pad_to_max_tokens=False)


In [27]:
# Find the average number of tokens in the training tweets
sum = 0
for i in train_sentences:
  sum = sum + len(i.split())
round(sum)
average = round(sum/len(train_sentences))
average

15

In [30]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                       output_sequence_length=max_length,
                                       output_mode="int")

In [31]:
# Fit the text vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [33]:
# Create a samplesentence and tokenize it
sample_sentence= "My name is sameer"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 13, 735,   9,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [41]:
# Vectorize a random sentence from training set
random_sentence = random.choice(train_sentences)

random_sentence, text_vectorizer([random_sentence])

("VIDEO: 'We're picking up bodies from water': Rescuers are searching for hundreds of migrants in the Mediterranean after a boat carryi...",
 <tf.Tensor: shape=(1, 15), dtype=int64, numpy=
 array([[ 72,  67, 728,  27, 713,  20, 190, 341,  22, 669,  10, 482,   6,
         330,   4]])>)

In [42]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5]

['', '[UNK]', 'the', 'a', 'in']

In [44]:
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [45]:
from tensorflow.keras import layers

In [47]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)
embedding

In [59]:
# turn sentences into dense vector of fixed size
random_sentence = random.choice(train_sentences)

sample_embed = embedding(text_vectorizer([random_sentence]))

random_sentence, sample_embed

('The 1st time someone blew up my phone 30 times they would be blocked. Believe it.  #Catfish',
 <tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[-0.02685957, -0.03080517,  0.02918035, ...,  0.03240104,
           0.00635713, -0.04190356],
         [-0.01509138,  0.01604826,  0.01223372, ...,  0.00265713,
          -0.01252048, -0.01793432],
         [-0.0138314 , -0.032279  , -0.04438777, ..., -0.01701869,
           0.03050486,  0.04014689],
         ...,
         [-0.01591193, -0.0492851 , -0.02717621, ...,  0.01960199,
           0.00083675,  0.04196235],
         [-0.01902555, -0.03997222,  0.00824126, ..., -0.00774685,
           0.03982954,  0.01922565],
         [ 0.01618004, -0.03209539, -0.03981844, ..., -0.01378219,
           0.00779831, -0.00998688]]], dtype=float32)>)

In [61]:
# checkout single tokens embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-2.6859570e-02, -3.0805171e-02,  2.9180352e-02,  4.4149313e-02,
        -1.8269084e-02, -1.1323262e-02,  8.4857121e-03,  1.0335587e-02,
        -2.2263622e-02,  2.8672148e-02, -9.1390684e-04, -4.9862154e-03,
        -2.1045078e-02, -1.3359405e-02,  4.6336856e-02, -3.3977389e-02,
         3.5530139e-02, -4.6359695e-02,  1.9985173e-02, -4.0678870e-02,
         2.6725937e-02,  5.2471384e-03, -1.4476694e-02,  2.4292994e-02,
         5.0733313e-03, -1.2376703e-02, -1.5046060e-02, -1.4928687e-02,
        -4.9035955e-02, -9.3955286e-03, -3.6249973e-02,  6.1243288e-03,
         3.0003414e-03, -4.6361890e-02,  4.8455309e-02,  4.3506514e-02,
         2.4883304e-02,  6.6457279e-03, -1.5491329e-02, -4.9855985e-02,
         3.3085179e-02,  5.2198991e-03, -3.0762220e-02,  4.2414632e-02,
        -4.7236122e-02, -4.8773527e-02,  2.0107273e-02, -8.2265586e-05,
         3.7731614e-02,  2.7522054e-02, -4.6931755e-02, -2.0820618e-02,
         4.5509